In [ ]:
# Import necessary libraries
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget


from llm_retrieval.wrapper import main

# Load environment variables for Neo4j credentials
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Connect to the Neo4j database
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def run_query(query: str):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

natural_langauge_query = "Finns det några socialdemokrater som delar samma förnamn?"
result = run_query("""
MATCH (p:Party {partyName: "Socialdemokraterna"})-[:BELONGS_TO]->(person:Person)
WITH  split(person.name, " ")[0] AS firstName, count(person) AS nameCount
WHERE nameCount > 1
RETURN firstName, nameCount
""")
main(natural_langauge_query)
print(natural_langauge_query)
print(result)
print()



natural_langauge_query = "Finns det några deltagare i debatten som delar samma förnamn?"
result = run_query("""
MATCH (p:Party)-[:BELONGS_TO]->(person:Person)
WITH split(person.name, " ")[0] AS firstName, count(person) AS nameCount
WHERE nameCount > 1
RETURN firstName, nameCount
""")
print(natural_langauge_query)
print(result)
print()



natural_langauge_query = "Vilket parti har minst antal deltagare i debatten, och vilket har flest?"
result = run_query("""
MATCH (person:Person)-[:BELONGS_TO]->(party:Party)
WITH party.partyName AS partyName, count(person) AS participants
ORDER BY participants ASC
RETURN partyName, participants
LIMIT 1

UNION

MATCH (person:Person)-[:BELONGS_TO]->(party:Party)
WITH party.partyName AS partyName, count(person) AS participants
ORDER BY participants DESC
RETURN partyName, participants
LIMIT 1
""")
print(natural_langauge_query)
print(result)
print()



natural_langauge_query = "Vem har flest inlägg i debatten och vilket parti tillhör personen?"
result = run_query("""
MATCH (person:Person)-[:STATED]->()
WITH person, count(person) AS statedCount
ORDER BY statedCount DESC
LIMIT 1
RETURN person.name, statedCount, person.party
""")
print(natural_langauge_query)
print(result)
print()


natural_langauge_query = "Vilket parti har flest kvinnor i debatten, och hur många kvinnor respektive män har de?"
result = run_query("""
MATCH (person:Person)-[:BELONGS_TO]->(party:Party)
WHERE person.gender = 'woman'
WITH party.partyName AS partyName, count(person) AS womenCount
ORDER BY womenCount DESC
LIMIT 1
WITH partyName, womenCount
MATCH (person:Person)-[:BELONGS_TO]->(party:Party)
WHERE person.gender = 'man'
WITH partyName, womenCount, count(person) AS menCount
RETURN partyName, womenCount, menCount
""")
print(natural_langauge_query)
print(result)
print()

natural_langauge_query = "Finns det något parti som inte har några kvinnliga deltagare i debatten?"
result = run_query("""
MATCH (party:Party)
WHERE NOT EXISTS {
    MATCH (party)<-[:BELONGS_TO]-(person:Person)
    WHERE person.gender = 'woman'
}
RETURN party.partyName
                   """)
print(natural_langauge_query)
print(result)



driver.close()


NameError: name '__file__' is not defined

Potentially useful for visualizing some queries:

In [22]:
g = Neo4jGraphWidget(driver)
g.show_cypher("MATCH (p:Party {partyName: 'Centerpartiet'})-[r]-(connected) RETURN p, r, connected")


GraphWidget(layout=Layout(height='500px', width='100%'))